In [48]:
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:4.2.7-0u

In [49]:
import os
import gymnasium
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [50]:
environment_name = 'CartPole-v1'
env = gymnasium.make(environment_name)

In [51]:
episodes = 5
for episode in range(1, episodes+1):
  state = env.reset()
  done = False
  score = 0.0

  while not done:
    action = env.action_space.sample()
    n_state, reward, done, truncated, info = env.step(action)
    score += reward
  
  print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:26.0
Episode:2 Score:14.0
Episode:3 Score:13.0
Episode:4 Score:42.0
Episode:5 Score:14.0


In [52]:
log_path = os.path.join('Training', 'Logs')

In [53]:
log_path

'Training/Logs'

In [54]:
env = gymnasium.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [55]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 442  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 380         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007867973 |
|    clip_fraction        | 0.0875      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00228    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.91        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 51.9        |
-----------------------------------------
---

In [56]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [57]:
model.save(PPO_Path)

In [58]:
del model

In [59]:
model = PPO.load(PPO_Path)

In [60]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(500.0, 0.0)

In [44]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()
  done = False
  score = 0.0

  while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    score += reward
  
  print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:[500.]
Episode:2 Score:[453.]
Episode:3 Score:[500.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


In [61]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [62]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [63]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gymnasium.make("CartPole-v1", render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

In [64]:
record_video("CartPole-v1", model, video_length=500, prefix="ppo-cartpole")

Saving video to /content/videos/ppo-cartpole-step-0-to-step-500.mp4
Moviepy - Building video /content/videos/ppo-cartpole-step-0-to-step-500.mp4.
Moviepy - Writing video /content/videos/ppo-cartpole-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/ppo-cartpole-step-0-to-step-500.mp4


In [65]:
show_videos("videos", prefix="ppo")

In [66]:
training_log_path = os.path.join(log_path, 'PPO_2')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
training_log_path

'Training/Logs/PPO_2'

In [68]:
!tensorboard --logdir={training_log_path}

2023-04-18 00:48:30.021539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.1 at http://localhost:6006/ (Press CTRL+C to quit)


^C


In [ ]:
size = 10

class Network:
  def __init__(self, weights_1=None, weights_2=None, weights_3=None, biases=None):
    self.network_outputs = size * size * 0.5
    self.layer_1 = size * size
    self.layer_2 = size * size
    if weights_1 is None:
      weights_1_shape = (self.layer_1, size * size)
      self.weights_1 = np.random.normal(size=weights_1_shape)
    else:
      self.weights_1 = weights_1
    if weights_2 is None:
      weights_2_shape = (self.layer_2, self.layer_1)
      self.weights_2 = np.random.normal(size=weights_2_shape)
    else:
      self.weights_2 = weights_2
    if weights_3 is None:
      weights_3_shape = (self.network_outputs, self.layer_2)
      self.weights_3 = np.random.normal(size=weights_3_shape)
    else:
      self.weights_3 = weights_3
    if biases is None:
      self.biases = np.random.normal(size=(self.network_outputs))
    else:
      self.biases = biases

  def clone(self):
    return Network(np.copy(self.weights_1), np.copy(self.weights_2), np.copy(self.weights_3), np.copy(self.biases), self.type)

  def forward(self, observations):
    outputs_sub1 = np.matmul(self.weights_1, observations)
    outputs_sub2 = np.matmul(self.weights_2, outputs_sub1)
    outputs = np.add(np.matmul(self.weights_3, outputs_sub2), self.biases)
    return outputs

  def copy_and_mutate(self, network, mr=0.1):
    self.weights_1 = np.add(
        network.weights_1, np.random.normal(size=self.weights_1.shape) * mr)
    self.weights_2 = np.add(
        network.weights_2, np.random.normal(size=self.weights_2.shape) * mr)
    self.weights_3 = np.add(
        network.weights_3, np.random.normal(size=self.weights_3.shape) * mr)
    self.biases = np.add(
        network.biases, np.random.normal(size=self.biases.shape) * mr)

  def copy(self, network):
    self.weights_1 = network.weights_1
    self.weights_2 = network.weights_2
    self.weights_3 = network.weights_3
    self.biases = network.biases

In [ ]:
from google.colab import files

